<a href="https://colab.research.google.com/github/basharbme/3D-Pelvic-Bone-cancer-segmentation-and-classification/blob/main/Pelvic_Bone_Tumour_3D_Classification_Training_Part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install monailabel

     |████████████████████████████████| 6.2 MB 5.4 MB/s 
     |████████████████████████████████| 650 kB 41.7 MB/s 
     |████████████████████████████████| 74 kB 2.7 MB/s 
     |████████████████████████████████| 10.1 MB 48.0 MB/s 
     |████████████████████████████████| 51 kB 6.0 MB/s 
     |████████████████████████████████| 2.7 MB 34.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 1.9 MB 39.3 MB/s 
     |████████████████████████████████| 1.6 MB 38.3 MB/s 
     |████████████████████████████████| 636 kB 46.8 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 60 kB 6.8 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 221 kB 41.9 MB/s 
     |████████████

In [ ]:
!pip install monai-deploy-app-sdk

     |████████████████████████████████| 119 kB 5.3 MB/s 
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.7.1
    Uninstalling typeguard-2.7.1:
      Successfully uninstalled typeguard-2.7.1


In [ ]:
import logging
import os
import sys

import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import monai
from monai.data import decollate_batch
from monai.metrics import ROCAUCMetric
from monai.transforms import Activations, AddChanneld, AsDiscrete, Compose, LoadImaged, RandRotate90d, Resized, ScaleIntensityd, EnsureTyped, EnsureType


def main():
    monai.config.print_config()
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

   
    data_path = os.sep.join(["/content/drive/My Drive/Data_folder/CT"])
    images = [
        "1.nii.gz",
        "2.nii.gz",
        "3.nii.gz",
        "4.nii.gz",
        "5.nii.gz",
        "6.nii.gz",
        "7.nii.gz",
        "8.nii.gz",
        "26.nii.gz",
        "28.nii.gz",
        "29.nii.gz",
    ]
    images = [os.sep.join([data_path, f]) for f in images]

     # 2 binary labels for tumour classification: primary and secondary
    labels = np.array([0,1], dtype=np.int8)
    train_files = [{"img": img, "label": label} for img, label in zip(images[:10], labels[:10])]
    val_files = [{"img": img, "label": label} for img, label in zip(images[-10:], labels[-10:])]

    # Define transforms for image
    train_transforms = Compose(
        [
            LoadImaged(keys=["img"]),
            AddChanneld(keys=["img"]),
            ScaleIntensityd(keys=["img"]),
            Resized(keys=["img"], spatial_size=(96, 96, 96)),
            RandRotate90d(keys=["img"], prob=0.8, spatial_axes=[0, 2]),
            EnsureTyped(keys=["img"]),
        ]
    )
    val_transforms = Compose(
        [
            LoadImaged(keys=["img"]),
            AddChanneld(keys=["img"]),
            ScaleIntensityd(keys=["img"]),
            Resized(keys=["img"], spatial_size=(96, 96, 96)),
            EnsureTyped(keys=["img"]),
        ]
    )
    post_pred = Compose([EnsureType(), Activations(softmax=True)])
    post_label = Compose([EnsureType(), AsDiscrete(to_onehot=True, num_classes=2)])

    # Define dataset, data loader
    check_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    check_loader = DataLoader(check_ds, batch_size=2, num_workers=4, pin_memory=torch.cuda.is_available())
    check_data = monai.utils.misc.first(check_loader)
    print(check_data["img"].shape, check_data["label"])

    # create a training data loader
    train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4, pin_memory=torch.cuda.is_available())

    # create a validation data loader
    val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=2, num_workers=4, pin_memory=torch.cuda.is_available())

    # Create DenseNet121, CrossEntropyLoss and Adam optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=2).to(device)
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), 1e-5)
    auc_metric = ROCAUCMetric()

    # start a typical PyTorch training
    val_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    writer = SummaryWriter()
    for epoch in range(5):
        print("-" * 10)
        print(f"epoch {epoch + 1}/{5}")
        model.train()
        epoch_loss = 0
        step = 0
        for batch_data in train_loader:
            step += 1
            inputs, labels = batch_data["img"].to(device), batch_data["label"].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_len = len(train_ds) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)
        epoch_loss /= step
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                y_pred = torch.tensor([], dtype=torch.float32, device=device)
                y = torch.tensor([], dtype=torch.long, device=device)
                for val_data in val_loader:
                    val_images, val_labels = val_data["img"].to(device), val_data["label"].to(device)
                    y_pred = torch.cat([y_pred, model(val_images)], dim=0)
                    y = torch.cat([y, val_labels], dim=0)

                acc_value = torch.eq(y_pred.argmax(dim=1), y)
                acc_metric = acc_value.sum().item() / len(acc_value)
                y_onehot = [post_label(i) for i in decollate_batch(y)]
                y_pred_act = [post_pred(i) for i in decollate_batch(y_pred)]
                auc_metric(y_pred_act, y_onehot)
                auc_result = auc_metric.aggregate()
                auc_metric.reset()
                del y_pred_act, y_onehot
                if acc_metric > best_metric:
                    best_metric = acc_metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), "best_metric_model_classification3d_dict.pth")
                    print("saved new best metric model")
                print(
                    "current epoch: {} current accuracy: {:.4f} current AUC: {:.4f} best accuracy: {:.4f} at epoch {}".format(
                        epoch + 1, acc_metric, auc_result, best_metric, best_metric_epoch
                    )
                )
                writer.add_scalar("val_accuracy", acc_metric, epoch + 1)
    print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
    writer.close()


if __name__ == "__main__":
    main()

MONAI version: 0.7.0
Numpy version: 1.19.5
Pytorch version: 1.9.0+cu111
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: bfa054b9c3064628a21f4c35bbe3132964e91f43

Optional dependencies:
Pytorch Ignite version: 0.4.5
Nibabel version: 3.0.2
scikit-image version: 0.16.2
Pillow version: 7.1.2
Tensorboard version: 2.6.0
gdown version: 3.6.4
TorchVision version: 0.10.0+cu111
tqdm version: 4.62.3
lmdb version: 0.99
psutil version: 5.4.8
pandas version: 1.1.5
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


torch.Size([2, 1, 96, 96, 96]) tensor([0, 0])
----------
epoch 1/5
1/5, train_loss: 0.6518
2/5, train_loss: 0.7352
3/5, train_loss: 0.7372
4/5, train_loss: 0.7495
5/5, train_loss: 0.6572
epoch 1 average loss: 0.7062
----------
epoch 2/5
1/5, train_loss: 0.7359
2/5, train_loss: 0.7186
3/5, train_loss: 0.7250
4/5, train_loss: 0.7511
5/5, train_loss: 0.6854
epoch 2 average loss: 0.7232
saved new best metric model
current epoch: 2 current accuracy: 0.4000 current AUC: 0.7500 best accuracy: 0.4000 at epoch 2
----------
epoch 3/5
1/5, train_loss: 0.7211
2/5, train_loss: 0.7111
3/5, train_loss: 0.7249
4/5, train_loss: 0.6883
5/5, train_loss: 0.6933
epoch 3 average loss: 0.7077
----------
epoch 4/5
1/5, train_loss: 0.7088
2/5, train_loss: 0.7312
3/5, train_loss: 0.6127
4/5, train_loss: 0.6833
5/5, train_loss: 0.6860
epoch 4 average loss: 0.6844
current epoch: 4 current accuracy: 0.4000 current AUC: 0.2083 best accuracy: 0.4000 at epoch 2
----------
epoch 5/5
1/5, train_loss: 0.6945
2/5, train_

In [ ]:
!pip install SimpleITK==2.1.0
!pip install torchsummaryX
!pip install nibabel
!pip install pillow
!pip install tensorboard
!pip install gdown
!pip install pytorch-ignite==0.4.4
!pip install itk
!pip install tqdm
!pip install lmdb
!pip install psutil
!pip install pandas
!pip install einops
!pip install scikit-image
!pip install 'monai[all]'

     |████████████████████████████████| 48.4 MB 36 kB/s 
  Attempting uninstall: SimpleITK
    Found existing installation: SimpleITK 2.1.1
    Uninstalling SimpleITK-2.1.1:
      Successfully uninstalled SimpleITK-2.1.1
     |████████████████████████████████| 200 kB 16.1 MB/s 
  Attempting uninstall: pytorch-ignite
    Found existing installation: pytorch-ignite 0.4.5
    Uninstalling pytorch-ignite-0.4.5:
      Successfully uninstalled pytorch-ignite-0.4.5
     |████████████████████████████████| 2.9 MB 10.9 MB/s 
  Using cached pytorch_ignite-0.4.5-py3-none-any.whl (221 kB)
     |████████████████████████████████| 4.9 MB 34.8 MB/s 
     |████████████████████████████████| 3.3 MB 44.3 MB/s 
     |████████████████████████████████| 895 kB 35.3 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
  Attempting uninstall: pytorch-ignite
    Found existing installation: pytorch-ignite 0.4.4
    Uninstalling pytorch-ignite-0.4.4:
      Successfully uninstalled pytorch-ignite-0.4.4
